In [229]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier as rfc
from sklearn.svm import LinearSVC
from sklearn import cross_validation as cv
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import log_loss,accuracy_score
from scipy.sparse import csr_matrix,hstack
import time

In [2]:
events = pd.read_csv('input/events.csv',index_col='event_id')
label = pd.read_csv('input/label_categories.csv')
app_event = pd.read_csv('input/app_events.csv')
app_label = pd.read_csv('input/app_labels.csv')
device = pd.read_csv('input/phone_brand_device_model.csv')
device = device.drop_duplicates('device_id').set_index('device_id')
gender_age_train = pd.read_csv('input/gender_age_train.csv',index_col = 'device_id')
gender_age_test = pd.read_csv('input/gender_age_test.csv',index_col = 'device_id')

For the convinience of creating sparse matrix, we create another integer index for the training and testing set.

In [3]:
gender_age_train['int_index'] = np.arange(gender_age_train.shape[0])
gender_age_test['int_index'] = np.arange(gender_age_test.shape[0])

In [4]:
print('gender_age_train\n',gender_age_train.head(1))
print('\ngender_age_test\n',gender_age_test.head(1))
print('\ndevice\n',device.head(1))
print('\nevents\n',events.head(1))
print('\nlabel\n',label.head(1))
print('\napp_event\n',app_event.head(1))
print('\napp_label\n',app_label.head(1))

gender_age_train
                      gender  age   group  int_index
device_id                                          
-8076087639492063270      M   35  M32-38          0

gender_age_test
                      int_index
device_id                     
1002079943728939269          0

device
                      phone_brand device_model
device_id                                    
-8890648629457979026          小米           红米

events
                   device_id            timestamp  longitude  latitude
event_id                                                             
1         29182687948017175  2016-05-01 00:55:25     121.38     31.24

label
    label_id category
0         1      NaN

app_event
    event_id               app_id  is_installed  is_active
0         2  5927333115845830913             1          1

app_label
                 app_id  label_id
0  7324884708820027918       251


# Feature engineering I: phone brand

Encoding the phone brand, and feed this as a feature into the training and testing test.

In [5]:
encoder = LabelEncoder()
encoder.fit(device.phone_brand)
device['brand'] = encoder.transform(device.phone_brand)
gender_age_train['brand'] = device['brand']
gender_age_test['brand'] = device['brand']

Create sparse matrix with each row representing one device and each column representing one brand

In [6]:
X_train_brand = csr_matrix((np.ones(gender_age_train.shape[0]),
                            (gender_age_train.int_index,gender_age_train.brand)))
X_test_brand = csr_matrix((np.ones(gender_age_test.shape[0]),
                           (gender_age_test.int_index,gender_age_test.brand)))
print('X_train_brand shape:',X_train_brand.shape)
print('X_test_brand shape:',X_test_brand.shape)

X_train_brand shape: (74645, 131)
X_test_brand shape: (112071, 131)


# Feature engineering II: Installed app

Encoding the app_id and store it into app column, and feed this as a feature into the training and testing test.

In [7]:
encoder2 = LabelEncoder()
encoder2.fit(app_event.app_id)
app_event['app'] = encoder2.transform(app_event.app_id)

In [8]:
np.sort(app_event.app.unique())

array([    0,     1,     2, ..., 19234, 19235, 19236], dtype=int64)

In [9]:
print(app_event.head(1))
print(events.head(1))

   event_id               app_id  is_installed  is_active    app
0         2  5927333115845830913             1          1  15408
                  device_id            timestamp  longitude  latitude
event_id                                                             
1         29182687948017175  2016-05-01 00:55:25     121.38     31.24


In [10]:
installed_app = pd.merge(events[['device_id']],app_event[['event_id','app','is_installed']],
                         how='right',right_on = 'event_id',left_index = True)
print(installed_app.head())

             device_id  event_id    app  is_installed
0 -6401643145415154744         2  15408             1
1 -6401643145415154744         2   3384             1
2 -6401643145415154744         2   7620             1
3 -6401643145415154744         2   8902             1
4 -6401643145415154744         2  18686             1


In [11]:
installed_app_grouped = installed_app.groupby(['device_id','app'])['app'].agg(['size'])
print('installed_app_grouped:')
print(installed_app_grouped.head())
installed_app_train = pd.merge(installed_app_grouped,gender_age_train[['int_index']],
                               how = 'right',right_index=True,left_index=True)
installed_app_test = pd.merge(installed_app_grouped,gender_age_test[['int_index']],
                              how = 'right',right_index=True,left_index=True)
print('installed_app_train:')
print(installed_app_train.head())

installed_app_grouped:
                           size
device_id            app       
-9222956879900151005 548     18
                     1096    18
                     1248    26
                     1545    12
                     1664    18
installed_app_train:
                           size  int_index
device_id            app                  
-9222956879900151005 548     18      21594
                     1096    18      21594
                     1248    26      21594
                     1545    12      21594
                     1664    18      21594


In [12]:
installed_app_train = installed_app_train.reset_index()
installed_app_test = installed_app_test.reset_index()
installed_app_train = installed_app_train.dropna(subset=['int_index'])
installed_app_test = installed_app_test.dropna(subset=['int_index'])
print(installed_app_train.head())
print(installed_app_test.head())

             device_id   app  size  int_index
0 -9222956879900151005   548    18      21594
1 -9222956879900151005  1096    18      21594
2 -9222956879900151005  1248    26      21594
3 -9222956879900151005  1545    12      21594
4 -9222956879900151005  1664    18      21594
             device_id    app  size  int_index
0 -9222661944218806987   1867     3      13612
1 -9222661944218806987   7519     8      13612
2 -9222661944218806987   7843     1      13612
3 -9222661944218806987   8704     4      13612
4 -9222661944218806987  10000     1      13612


Calculate the number of unique app:

In [13]:
print('The number of unique apps:')
print(np.size(installed_app.app.unique()))
appnumber = np.size(installed_app.app.unique())

The number of unique apps:
19237


In [14]:
print(gender_age_test.shape[0])
print(installed_app_train.shape[0])

112071
915632


In [15]:
print(np.sort(installed_app_train.app.unique()))
print(installed_app_test.shape[0])

[    0     1     2 ..., 19234 19235 19236]
1387337


In [16]:
X_train_installed = csr_matrix((np.ones(installed_app_train.shape[0]),
                                (installed_app_train.int_index,installed_app_train.app)), 
                               shape = (gender_age_train.shape[0],appnumber))
X_test_installed = csr_matrix((np.ones(installed_app_test.shape[0]),
                               (installed_app_test.int_index,installed_app_test.app)),
                               shape = (gender_age_test.shape[0],appnumber))
print('X_train_installed shape:',X_train_installed.shape)
print('X_test_installed shape:',X_test_installed.shape)

X_train_installed shape: (74645, 19237)
X_test_installed shape: (112071, 19237)


In [17]:
np.sort(installed_app_test.int_index.unique())

array([     0,      1,      2, ..., 112059, 112060, 112063], dtype=int64)

# Feature engineering III: phone device model

In [18]:
brand_model = device.phone_brand.str.cat(device.device_model)
encoder3 = LabelEncoder()
encoder3.fit(brand_model)
device['model'] = encoder3.transform(brand_model)
gender_age_train['model'] = device['model']
gender_age_test['model'] = device['model']

In [19]:
X_train_model = csr_matrix((np.ones(gender_age_train.shape[0]),(gender_age_train.int_index,gender_age_train.model)))
X_test_model = csr_matrix((np.ones(gender_age_test.shape[0]),(gender_age_test.int_index,gender_age_test.model)))
print('X_train_brand shape:',X_train_model.shape)
print('X_test_brand shape:',X_test_model.shape)

X_train_brand shape: (74645, 1667)
X_test_brand shape: (112071, 1667)


# Feature engineering IV: app label

In [20]:
print(app_event[['app_id','event_id']].head(1))
print(app_label[['app_id','label_id']].head(1))

                app_id  event_id
0  5927333115845830913         2
                app_id  label_id
0  7324884708820027918       251


In [21]:
app_label_new = app_label.loc[app_label.app_id.isin(app_event.app_id.unique())].copy()
app_label_new['app'] = encoder2.transform(app_label_new.app_id)
encoder4 = LabelEncoder().fit(app_label_new.label_id)
app_label_new['label'] = encoder4.transform(app_label_new.label_id)
labelnumber = len(encoder4.classes_)
print('app_label_new:')
print(app_label_new.head())

app_label_new:
                app_id  label_id    app  label
0  7324884708820027918       251  17355    207
1 -4494216993218550286       251   4618    207
2  6058196446775239644       406  15548    247
3  6058196446775239644       407  15548    248
4  8694625920731541625       406  18689    247


In [22]:
print(app_label.size)
print(installed_app.size)
print('installed_app_grouped:')
print(installed_app_grouped.head())
installed_label_grouped = (installed_app_grouped.reset_index()[['device_id','app']]
                          .merge(app_label_new[['app','label']])
                          .groupby(['device_id','label']))['app'].agg(['size']).reset_index()
                          
print('installed_label_grouped:')
print(installed_label_grouped.head())

919886
129892268
installed_app_grouped:
                           size
device_id            app       
-9222956879900151005 548     18
                     1096    18
                     1248    26
                     1545    12
                     1664    18
installed_label_grouped:
             device_id  label  size
0 -9222956879900151005    117     1
1 -9222956879900151005    120     1
2 -9222956879900151005    126     1
3 -9222956879900151005    138     2
4 -9222956879900151005    147     2


In [23]:
label_app_train = pd.merge(installed_label_grouped,gender_age_train[['int_index']],
                               how = 'right',right_index=True,left_on='device_id')
label_app_test = pd.merge(installed_label_grouped,gender_age_test[['int_index']],
                              how = 'right',right_index=True,left_on ='device_id' )
label_app_train = label_app_train.dropna(subset= ['int_index','label'])
label_app_test = label_app_test.dropna(subset= ['int_index','label'])

In [24]:
X_train_label = csr_matrix((np.ones(label_app_train.shape[0]),(label_app_train.int_index,label_app_train.label)),
                              shape = (gender_age_train.shape[0],labelnumber))
X_test_label = csr_matrix((np.ones(label_app_test.shape[0]),(label_app_test.int_index,label_app_test.label)),
                              shape = (gender_age_test.shape[0],labelnumber))
print('X_train_installed shape:',X_train_label.shape)
print('X_test_installed shape:',X_test_label.shape)

X_train_installed shape: (74645, 492)
X_test_installed shape: (112071, 492)


# Feature engineering V: active app

In [25]:
active_app = pd.merge(events[['device_id']],app_event[['event_id','app','is_active']],
                         how='right',right_on = 'event_id',left_index = True)
active_app = active_app.loc[active_app.is_active==1]
print(active_app.head())

             device_id  event_id    app  is_active
0 -6401643145415154744         2  15408          1
3 -6401643145415154744         2   8902          1
4 -6401643145415154744         2  18686          1
5 -6401643145415154744         2  14346          1
9 -6401643145415154744         2  16908          1


## tried to use a TF-IDF like factor to suppress the very common app influence

In [149]:
#use TF-IDF
active_app_count = active_app.groupby(['device_id','app'])['device_id'].agg({'usage_number':'size'}).reset_index()
active_app_count = active_app_count.groupby(['app'])['usage_number'].agg({'device_number':'count'}).reset_index()
total_user  = gender_age_test.shape[0] + gender_age_train.shape[0]
active_app_count['IDF'] = np.log((total_user)/(active_app_count.device_number))
active_app_count.head()

,app,device_number,IDF
0,0,1,12.137344
1,5,1,12.137344
2,6,63,7.994209
3,7,24,8.959290
4,8,1,12.137344


In [153]:
active_app_grouped = active_app.groupby(['device_id','app'])['app'].agg(['size']).reset_index()
print('active_app_grouped:')
print(active_app_grouped.head())
active_app_grouped = pd.merge(active_app_grouped,active_app_count[['app','IDF']]).set_index('device_id')
print('active_app_grouped:')
print(active_app_grouped.head())

active_app_grouped:
             device_id   app  size
0 -9222956879900151005   548     4
1 -9222956879900151005  1248    15
2 -9222956879900151005  1545     2
3 -9222956879900151005  1848    31
4 -9222956879900151005  2236    17
active_app_grouped:
                      app  size       IDF
device_id                                
-9222956879900151005  548     4  6.329202
-9221825537663503111  548     2  6.329202
-9211913362970025570  548     3  6.329202
-9174782581415497963  548    10  6.329202
-9102874319378078549  548     1  6.329202


In [157]:
active_app_train = pd.merge(active_app_grouped,gender_age_train[['int_index']],
                               how = 'left',right_index=True,left_index=True)
active_app_test = pd.merge(active_app_grouped,gender_age_test[['int_index']],
                              how = 'left',right_index=True,left_index=True)
print('active_app_train:')
print(active_app_train.head())

active_app_train:
                       app  size       IDF  int_index
device_id                                            
-9222956879900151005   548     4  6.329202    21594.0
-9222956879900151005  1248    15  6.879849    21594.0
-9222956879900151005  1545     2  5.757221    21594.0
-9222956879900151005  1848    31  4.854583    21594.0
-9222956879900151005  2236    17  4.852523    21594.0


In [158]:
active_app_train = active_app_train.reset_index()
active_app_test = active_app_test.reset_index()
active_app_train = active_app_train.dropna(subset=['int_index'])
active_app_test = active_app_test.dropna(subset=['int_index'])
print(active_app_train.head())
print(active_app_test.head())

             device_id   app  size       IDF  int_index
0 -9222956879900151005   548     4  6.329202    21594.0
1 -9222956879900151005  1248    15  6.879849    21594.0
2 -9222956879900151005  1545     2  5.757221    21594.0
3 -9222956879900151005  1848    31  4.854583    21594.0
4 -9222956879900151005  2236    17  4.852523    21594.0
              device_id    app  size       IDF  int_index
55 -9222661944218806987  13919     2  1.971800    13612.0
56 -9222661944218806987  15408     2  1.778586    13612.0
57 -9222661944218806987  18686     6  1.578411    13612.0
58 -9222661944218806987   1867     3  5.000861    13612.0
59 -9222661944218806987   7519     7  4.608475    13612.0


In [159]:
print(np.sort(active_app_train.app.unique()))
print(appnumber)

[    0     5     6 ..., 19225 19228 19236]
19237


In [202]:
appnumber = np.size(installed_app.app.unique())
# binary active map
#X_train_active = csr_matrix((np.ones(active_app_train.shape[0]),
#                            (active_app_train.int_index,active_app_train.app)), 
#                            shape = (gender_age_train.shape[0],appnumber))
#X_test_active = csr_matrix((np.ones(active_app_test.shape[0]),
#                            (active_app_test.int_index,active_app_test.app)),
#                            shape = (gender_age_test.shape[0],appnumber))
# count the number of active app
X_train_active = csr_matrix((np.log(np.log(active_app_train['size']*active_app_train['IDF']+1)+1),
                            (active_app_train.int_index,active_app_train.app)), 
                            shape = (gender_age_train.shape[0],appnumber))
X_test_active = csr_matrix((np.log(np.log(active_app_test['size']*active_app_test['IDF']+1)+1),
                            (active_app_test.int_index,active_app_test.app)),
                            shape = (gender_age_test.shape[0],appnumber))
print('X_train_active shape:',X_train_active.shape)
print('X_test_active shape:',X_test_active.shape)

X_train_active shape: (74645, 19237)
X_test_active shape: (112071, 19237)


# Feature engineering VI: active time period

# Model training and fitting: Logistic Regression

In [203]:
X_train_total = hstack((X_train_brand,X_train_model,X_train_installed,X_train_label,X_train_active),format='csr')
X_test_total = hstack((X_test_brand,X_test_model,X_test_installed,X_test_label,X_test_active),format='csr')
print('Training shape:')
print(X_train_total.shape)
print('Testing shape:')
print(X_test_total.shape)

Training shape:
(74645, 40764)
Testing shape:
(112071, 40764)


In [204]:
targetencoder = LabelEncoder()
targetencoder.fit(gender_age_train.group)
y_train_total = targetencoder.transform(gender_age_train.group)
target_len = len(targetencoder.classes_)
print(target_len)

12


In [205]:
X_train,X_test,y_train,y_test = cv.train_test_split(X_train_total,y_train_total)

In [225]:
model = LogisticRegression(C=0.02,multi_class='multinomial',solver='newton-cg')
model.fit(X_train,y_train)

LogisticRegression(C=0.02, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=200, multi_class='multinomial',
          n_jobs=1, penalty='l2', random_state=None, solver='newton-cg',
          tol=0.0001, verbose=0, warm_start=False)

In [226]:
y_pred = model.predict_proba(X_test)

In [227]:
log_loss(y_test,y_pred)

2.2804717764871203

In [228]:
model.score(X_test,y_test)

0.1919408423534455

In [179]:
model.fit(X_train_total,y_train_total)
pred = pd.DataFrame(model.predict_proba(X_test_total), index = gender_age_test.index, columns=targetencoder.classes_)
pred.head()

,F23-,F24-26,F27-28,F29-32,F33-42,F43+,M22-,M23-26,M27-28,M29-31,M32-38,M39+
device_id,,,,,,,,,,,,
1002079943728939269,0.001481,0.006373,0.011676,0.013228,0.026906,0.046867,0.012598,0.030805,0.070068,0.120400,0.275928,0.383670
-1547860181818787117,0.005798,0.010950,0.026181,0.061898,0.070445,0.154595,0.004471,0.077990,0.056872,0.070562,0.260329,0.199908
7374582448058474277,0.021069,0.038822,0.036541,0.174594,0.173732,0.065865,0.012815,0.029650,0.039598,0.106142,0.174044,0.127127
-6220210354783429585,0.002862,0.026509,0.007024,0.011298,0.051302,0.156133,0.049900,0.132713,0.094730,0.108318,0.175350,0.183862
-5893464122623104785,0.047285,0.065402,0.042666,0.062407,0.056764,0.043209,0.092214,0.165119,0.097584,0.101636,0.132676,0.093037


In [180]:
pred.to_csv('logreg_submission.csv',index=True)

# Model fitting :RF

In [234]:
model2 = rfc(n_estimators=300,verbose=1,n_jobs=-1)
model2.fit(X_train,y_train)

[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:   53.4s
[Parallel(n_jobs=-1)]: Done 184 tasks      | elapsed:  4.2min
[Parallel(n_jobs=-1)]: Done 300 out of 300 | elapsed:  6.8min finished


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=300, n_jobs=-1,
            oob_score=False, random_state=None, verbose=1,
            warm_start=False)

In [236]:
y_pred2 = model2.predict_proba(X_test)

[Parallel(n_jobs=8)]: Done  34 tasks      | elapsed:    0.0s
[Parallel(n_jobs=8)]: Done 184 tasks      | elapsed:    0.3s
[Parallel(n_jobs=8)]: Done 300 out of 300 | elapsed:    0.6s finished


In [237]:
log_loss(y_test,y_pred2)

3.2819344751732902